In [ ]:
import fisher 
import foregrounds_fisher as fg
import spectral_distortions as sd
import numpy as np
from noise_funcs import *
from help_funcs import *

# original version of sd_foregrounds 
(i.e. example.py)

In [ ]:
# PIXIE with all signals is the default. 
fish = fisher.FisherEstimation()
# args are stored in fish.args, values are stored in dictionary fish.argvals, 
# and fisher uncertainties are in fish.errors
fish.run_fisher_calculation()
# print the errors in sigma
fish.print_errors()

# To set the signals by hand, just modify the fncs arg here:
fish = fisher.FisherEstimation()
fish.set_signals(fncs=[sd.DeltaI_reltSZ_2param_yweight, sd.DeltaI_DeltaT, sd.DeltaI_mu,
                       fg.thermal_dust_rad, fg.cib_rad, fg.jens_freefree_rad,
                       fg.jens_synch_rad, fg.spinning_dust, fg.co_rad])
fish.run_fisher_calculation()
fish.print_errors()


# To change the frequencies (Hz), duration (months), or scale the noise by mult, 
# turn off the step function bandpass or change fsky, 
# edit any of the following
fish = fisher.FisherEstimation(fmin=5.e9, fmax=1.e12, fstep=5.e9, duration=60, mult=0.1, bandpass=False, fsky=0.5)


# Lastly to put priors (in fractions of the parameter value), drop the first n bins or mask out Galactic CO lines do:
fish = fisher.FisherEstimation(priors={'Td':0.1, 'Asd':0.01}, drop=2, doCO=True) 


# Example for specter set-up


In [ ]:
# define some bands and detectors for each band and other variables 
# this example uses ithe 

optimized_data=np.loadtxt("data/mu_1to2000_width1_snr25_bolo10_optimized_bands.txt", delimiter=',')
BANDS=optimized_data[:,:2].reshape(len(optimized_data[:,0]),2)

print('Frequency Bands [Hz]:')
print(BANDS)
DETS=np.array([  2.,   4.,   6.,   6.,   8.,  20.,  100., 100., 100., 100.,100,100,100, 100., 100., 100.])
print('Detector Counts:')
print(DETS)

FSKY=0.7 
DUR_MONTHS=12
HEMT_AMPS=True
HEMT_FREQ=10
FILE_PREFIX="SPECTER"

In [ ]:
%%time 
#run fisher


fish = fisher.FisherEstimation(fsky=FSKY, #observed sky fraction/percentage of TOD
                               duration=DUR_MONTHS, #observation duration in months
                               bandpass=False, #assume center frequencies are just the midpoint of the band edges 
                               instrument='specter', # select specter (default is 'pixie')
                               file_prefix=FILE_PREFIX, # prefix for all the bolocalc output files
                               freq_bands=BANDS, #bands array (shape is (# of bands, 2))
                               Ndet_arr=DETS, #detector array (1-D array of length # of bands )
                               hemt_amps=HEMT_AMPS, #use hemt amplifiers for low frequencies (recommended for anything below 10GHz)
                               hemt_freq=HEMT_FREQ, #below which frequency to use hempt amplifiers if hemt_amps=True
                               noisefile=None, #use instantanous sensitivity file in muK-rt(s) or compute sensitity using bolocalc
                               priors={}, #priors on model parameters
                               systematic_error=[], #compute bias for a given systematic error
                               arg_dict={}) #change fiducial parameters
fish.run_fisher_calculation()
fish.print_errors()

## Precompute instantanous noise to avoid running bolocalc each time


In [ ]:
print("Precomputing instantanous noise")
specter_freqs, specter_noise=getnoise_raw(path="data/", 
                                          bands=BANDS, 
                                          prefix=FILE_PREFIX,
                                          hemt_amps=HEMT_AMPS,
                                          hemt_freq=HEMT_FREQ)
print(f"Saved {FILE_PREFIX}_raw_sensitivity.txt file in /data")

In [ ]:
%%time
print("Now we can compute fisher super fast")

fish = fisher.FisherEstimation(fsky=FSKY, #observed sky fraction/percentage of TOD
                               duration=DUR_MONTHS, #observation duration in months
                               bandpass=False, #assume center frequencies are just the midpoint of the band edges 
                               instrument='specter', # select specter (default is 'pixie')
                               file_prefix=FILE_PREFIX, # prefix for all the bolocalc output files
                               freq_bands=BANDS, #bands array (shape is (# of bands, 2))
                               Ndet_arr=DETS, #detector array (1-D array of length # of bands )
                               hemt_amps=HEMT_AMPS, #use hemt amplifiers for low frequencies (recommended for anything below 10GHz)
                               hemt_freq=HEMT_FREQ, #below which frequency to use hempt amplifiers if hemt_amps=True
                               noisefile="data/SPECTER_raw_sensitivity.txt", #use instantanous sensitivity file in muK-rt(s) or compute sensitity using bolocalc
                               priors={}, #priors on model parameters
                               systematic_error=[], #compute bias for a given systematic error
                               arg_dict={}) #change fiducial parameters
fish.run_fisher_calculation()
fish.print_errors()

# Example computing bias 


In [ ]:
# 3 calibrators with (10**-3 muK_RJ calibration error) example

SYS_ERR_nano=muKtoJypersr_RJ(10**-3, specter_freqs*1e9) 
SYS_ERR_nano[-6:]*=-1.0 #6 highest frequencies are systematically biased in opposite direction


fish = fisher.FisherEstimation(fsky=FSKY, #observed sky fraction/percentage of TOD
                               duration=DUR_MONTHS, #observation duration in months
                               bandpass=False, #assume center frequencies are just the midpoint of the band edges 
                               instrument='specter', # select specter (default is 'pixie')
                               file_prefix=FILE_PREFIX, # prefix for all the bolocalc output files
                               freq_bands=BANDS, #bands array (shape is (# of bands, 2))
                               Ndet_arr=DETS, #detector array (1-D array of length # of bands )
                               hemt_amps=HEMT_AMPS, #use hemt amplifiers for low frequencies (recommended for anything below 10GHz)
                               hemt_freq=HEMT_FREQ, #below which frequency to use hempt amplifiers if hemt_amps=True
                               noisefile="data/SPECTER_raw_sensitivity.txt", #use instantanous sensitivity file in muK-rt(s) or compute sensitity using bolocalc
                               priors={}, #priors on model parameters
                               systematic_error=SYS_ERR_nano, #compute bias for a given systematic error
                               arg_dict={}) #change fiducial parameters

fish.run_fisher_calculation()
fish.calculate_systematic_bias()
mufid=2.e-8
print("assuming 10^-3 muK_RJ systematic error:")
print(f"percent bias: {fish.B['mu_amp']/2e-8*100.}")
print(f"sigma bias: {fish.B['mu_amp']/fish.errors['mu_amp']}")


# Example computing fisher errors using different fiducial values for the model

In [ ]:
#test increasing fiducial foreground spectral parameters values by 5% 
# (dust index & temperature, cib index & temperature, synchrotron index & curvature)

fid_params_keys=np.array(['Bd' ,'Td' ,'Bcib', 'Tcib', 'alps', 'w2s'])
factors=np.array([1.05, 1.05, 1.05, 1.05, 1.05, 1.05]) 

#this function outputs a dictionary with the above parameters 
# while keeping the rest to their fiducial values
test_param_dict=convert_one_dict(factors,fid_params_keys, factor_one=True)


fish = fisher.FisherEstimation(fsky=FSKY, #observed sky fraction/percentage of TOD
                               duration=DUR_MONTHS, #observation duration in months
                               bandpass=False, #assume center frequencies are just the midpoint of the band edges 
                               instrument='specter', # select specter (default is 'pixie')
                               file_prefix=FILE_PREFIX, # prefix for all the bolocalc output files
                               freq_bands=BANDS, #bands array (shape is (# of bands, 2))
                               Ndet_arr=DETS, #detector array (1-D array of length # of bands )
                               hemt_amps=HEMT_AMPS, #use hemt amplifiers for low frequencies (recommended for anything below 10GHz)
                               hemt_freq=HEMT_FREQ, #below which frequency to use hempt amplifiers if hemt_amps=True
                               noisefile="data/SPECTER_raw_sensitivity.txt", #use instantanous sensitivity file in muK-rt(s) or compute sensitity using bolocalc
                               priors={}, #priors on model parameters
                               systematic_error=[], #compute bias for a given systematic error
                               arg_dict=test_param_dict) #change fiducial parameters
fish.run_fisher_calculation()
fish.print_errors()
